# 均值回歸策略探索

本 Notebook 用於探索和測試均值回歸策略 (Mean Reversion Strategy)

使用 RSI 和 ATR 指標進行交易

In [ ]:
# 導入必要的套件
import sys
import os

# 加入專案根目錄到路徑
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

import pandas as pd
import matplotlib.pyplot as plt

from utils.fetch import fetch_stock_data, save_stock_data, load_stock_data
from strategies.mean_reversion import MeanReversionStrategy
from backtest.backtester import Backtester
from utils.plot import plot_backtest_results, plot_rsi

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

## 1. 載入資料

In [ ]:
# 設定股票代碼
symbol = '0050'  # 元大台灣50

# 抓取資料
df = fetch_stock_data(symbol, source='yfinance')

print(f"資料筆數: {len(df)}")
df.head()

## 2. 計算 RSI 和 ATR 指標

In [ ]:
# 建立均值回歸策略實例
strategy = MeanReversionStrategy(
    rsi_period=14,
    rsi_oversold=30,
    rsi_overbought=70,
    atr_period=14,
    atr_multiplier=2.0
)

# 計算指標
df_with_indicators = strategy.calculate_indicators(df)

# 顯示最後幾筆資料
df_with_indicators[['close', 'RSI', 'ATR']].tail(10)

In [ ]:
# 繪製 RSI 指標
plot_rsi(
    df_with_indicators[-252:],  # 最近一年
    title=f'{symbol} RSI 指標 (最近一年)'
)

## 3. 生成交易訊號

In [ ]:
# 生成交易訊號
df_signals = strategy.generate_signals(df)

# 查看有訊號的時間點
signals = df_signals[df_signals['signal'] != 0][['close', 'RSI', 'ATR', 'signal', 'stop_loss']]
print(f"交易訊號數量: {len(signals)}")
signals.tail(10)

## 4. 回測策略

In [ ]:
# 建立回測引擎
backtester = Backtester(initial_capital=100000, commission=0.001425)

# 執行回測
results = backtester.run(strategy, df)

# 顯示績效摘要
metrics = backtester.summary()

print("\n=== 回測結果 ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# 繪製回測結果
plot_backtest_results(results, title=f'{symbol} 均值回歸策略回測')

## 5. 參數優化

In [ ]:
from ai.optimize_strategy import StrategyOptimizer

# 建立優化器
optimizer = StrategyOptimizer(df, initial_capital=100000)

# 定義參數網格
param_grid = {
    'rsi_period': [10, 14, 20],
    'rsi_oversold': [25, 30, 35],
    'rsi_overbought': [65, 70, 75],
    'atr_period': [14],
    'atr_multiplier': [2.0]
}

# 執行網格搜尋
optimization_results = optimizer.grid_search(MeanReversionStrategy, param_grid)

In [ ]:
# 顯示最佳結果
best_result = optimization_results[0]

print("\n=== 最佳參數組合 ===")
print(f"參數: {best_result['parameters']}")
print(f"\n績效指標:")
for key, value in best_result['metrics'].items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# 比較兩種策略
from strategies.momentum import MomentumStrategy

momentum = MomentumStrategy(short_window=20, long_window=60)
mean_rev = MeanReversionStrategy(rsi_period=14, rsi_oversold=30, rsi_overbought=70)

# 回測兩種策略
bt1 = Backtester(initial_capital=100000)
bt2 = Backtester(initial_capital=100000)

results1 = bt1.run(momentum, df)
results2 = bt2.run(mean_rev, df)

metrics1 = bt1.summary()
metrics2 = bt2.summary()

print("\n=== 策略比較 ===")
print(f"\n動量策略:")
print(f"  總報酬率: {metrics1['total_return']:.2f}%")
print(f"  最大回撤: {metrics1['max_drawdown']:.2f}%")
print(f"  夏普比率: {metrics1['sharpe_ratio']:.2f}")

print(f"\n均值回歸策略:")
print(f"  總報酬率: {metrics2['total_return']:.2f}%")
print(f"  最大回撤: {metrics2['max_drawdown']:.2f}%")
print(f"  夏普比率: {metrics2['sharpe_ratio']:.2f}")